<a href="https://colab.research.google.com/github/Siri-1416/Sample/blob/master/Parkinson's_Disease_Detection.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#Importing the Libraries

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler
from sklearn import svm
from sklearn.metrics import accuracy_score
import warnings
warnings.filterwarnings('ignore')
import seaborn as sns

In [ ]:
from keras.utils.np_utils import to_categorical
from keras.models import Sequential
from keras.layers import Dense,Conv1D, MaxPooling1D, Flatten, Dense
from keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.optimizers import Adam
from sklearn.model_selection import train_test_split

#Data Collection & Analysis

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
# loading the data from csv file to a Pandas DataFrame
parkinsons_data = pd.read_csv('/content/drive/MyDrive/My project/parkinsons.csv')

In [ ]:
# printing the first 5 rows of the dataframe
parkinsons_data.head()

In [ ]:
# number of rows and columns in the dataframe
parkinsons_data.shape

In [ ]:
# getting more information about the dataset
parkinsons_data.info()

In [ ]:
# checking for missing values in each column
parkinsons_data.isnull().sum()

In [ ]:
# getting some statistical measures about the data
parkinsons_data.describe()

In [ ]:
# distribution of target Variable
parkinsons_data['status'].value_counts()

1  --> Parkinson's Positive

0 --> Healthy


In [ ]:

# grouping the data based on the target variable
parkinsons_data.groupby('status').mean()

In [ ]:
parkinsons_data.corr()

In [ ]:
sns.set(style="whitegrid")
plt.figure(figsize=(10,5))
ax=sns.countplot(x="status",data=parkinsons_data,palette=sns.color_palette("cubehelix",4))
plt.xticks(rotation=90)
plt.title("Class Label Counts",{"fontname":"fantasy","fontweight":"bold","fontsize":"medium"})
plt.ylabel("count", {"fontname":"serif","fontweight":"bold"})
plt.xlabel("Class", {"fontname":"serif","fontweight":"bold"})



In [ ]:
fig = plt.figure(figsize =(10, 7))

stat = parkinsons_data['status']

x = [0, 1]
l = ['0','1']
c = ['blue', 'orange']

#bar plot
plt.bar(x, height = 1000, color = c, label = l)
plt.grid(False)
plt.xticks([0,1])
plt.legend(loc=(1,0.5))

plt.title('0 : not affected & 1 : affected')
plt.show()

print(' ')

#pie chart
plt.pie([1000,1000],labels=['not affected','affect'],autopct='%.2f%%')
plt.legend(loc=(1,0.5))
plt.title('0 : not affected & 1 : affect')
plt.show()


#Data Preprocessing

##Separating the features & Target

In [ ]:
X = parkinsons_data.drop(columns=['name','status'], axis=1)
Y = parkinsons_data['status']

In [ ]:
X

In [ ]:
print(Y)

##Splitting the data to training data & Test data



In [ ]:
X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size=0.2, random_state=2)

In [ ]:
print(X.shape, X_train.shape, X_test.shape)

##Data Standardization

In [ ]:
scaler = MinMaxScaler()

In [ ]:
X_train = scaler.fit_transform(X_train)

X_test = scaler.transform(X_test)

In [ ]:
print(X_train)

#Model Training using Support Vector Machine Model

##Training model

In [ ]:
model = svm.SVC(kernel='linear', random_state = 10 )

In [ ]:
# training the SVM model with training data
model.fit(X_train, Y_train)

In [ ]:
Y_pred = model.predict(X_test)

##Model Evaluation

###Confusion Matrix

In [ ]:
from sklearn.metrics import confusion_matrix

cf = confusion_matrix(Y_test, Y_pred )

In [ ]:
cf

In [ ]:
import seaborn as sns
sns.heatmap(cf, annot=True)

###Accuracy Score

In [ ]:
# accuracy score on training data
X_train_prediction = model.predict(X_train)
training_data_accuracy = accuracy_score(Y_train, X_train_prediction)

In [ ]:
print('Accuracy score of training data : ', training_data_accuracy)

In [ ]:
# accuracy score on training data
X_test_prediction = model.predict(X_test)
test_data_accuracy = accuracy_score(Y_test, X_test_prediction)

In [ ]:
print('Accuracy score of test data : ', test_data_accuracy)

#Model Training using CNN

##Training model

In [ ]:
# Define the CNN model
cnn_model = Sequential()
cnn_model.add(Conv1D(64, 3, activation='relu', input_shape=(X_train.shape[1], 1)))
cnn_model.add(MaxPooling1D(2))
cnn_model.add(Conv1D(32, 3, activation='relu'))
cnn_model.add(MaxPooling1D(2))
cnn_model.add(Flatten())
cnn_model.add(Dense(16, activation='relu'))
cnn_model.add(Dense(1, activation='sigmoid'))

# Compile the model
cnn_model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])

In [ ]:
cnn_model.fit(X_train, Y_train, epochs=100, batch_size=16, verbose=0)

In [ ]:
# Evaluate the model on the testing set
loss, accuracy = cnn_model.evaluate(X_test, Y_test, verbose=0)
print(f'Test loss: {loss:.4f}\nTest accuracy: {accuracy:.4f}')

In [ ]:
from tensorflow.keras.utils import plot_model
plot_model(cnn_model,
           to_file="/content/drive/MyDrive/My project/model.png",
           show_shapes=True,
           show_layer_names=True,
          )

##Model Evaluation

###Confusion Matrix

In [ ]:
from sklearn.metrics import confusion_matrix
y_pred_cnn = cnn_model.predict(X_test)
cf_matrix_cnn = confusion_matrix(Y_test, y_pred_cnn.round())

In [ ]:
cf_matrix_cnn

In [ ]:
import seaborn as sns
sns.heatmap(cf_matrix_cnn, annot=True)

###Accuracy Score

In [ ]:
from sklearn.metrics import classification_report as cr
# accuracy score on training data
print('The Testing accuracy is : ', accuracy_score(Y_test, y_pred_cnn.round()))
print(cr(Y_test, y_pred_cnn.round()))


#Model Training using ANN

##Training model

In [ ]:
# Define the ANN model
classifier = Sequential()
classifier.add(Dense(16, input_dim=X_train.shape[1], activation='relu'))
classifier.add(Dense(8, activation='relu'))
classifier.add(Dense(1, activation='sigmoid'))

# Compile the model
classifier.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])

In [ ]:
classifier.fit(X_train, Y_train, epochs=100, batch_size=16, verbose=0)

In [ ]:
# Evaluate the model on the testing set
loss, accuracy = classifier.evaluate(X_test, Y_test, verbose=0)
print(f'Test loss: {loss:.4f}\nTest accuracy: {accuracy:.4f}')

In [ ]:
from tensorflow.keras.utils import plot_model
plot_model(classifier,
           to_file="/content/drive/MyDrive/My project/model.png",
           show_shapes=True,
           show_layer_names=True,
          )

##Model Evaluation

###Confusion Matrix

In [ ]:
from sklearn.metrics import confusion_matrix
y_pred_ann = classifier.predict(X_test)
cf_matrix_ann = confusion_matrix(Y_test, y_pred_ann.round())

In [ ]:
cf_matrix_ann

In [ ]:
import seaborn as sns
sns.heatmap(cf_matrix_ann, annot=True)

###Accuracy Score

In [ ]:
from sklearn.metrics import classification_report as cr
# accuracy score on training data
print('The Testing accuracy is : ', accuracy_score(Y_test, y_pred_ann.round()))
print(cr(Y_test, y_pred_ann.round()))


#Building a Predictive System

##Using Support Vector Machine

In [ ]:
input_data = (199.22800,209.51200,192.09100,0.00241,0.00001,0.00134,0.00138,0.00402,0.01015,0.08900,0.00504,0.00641,0.00762,0.01513,0.00167,30.94000,0.432439,0.742055,-7.682587,0.173319,2.103106,0.068501
)
# changing input data to a numpy array
input_data_as_numpy_array = np.asarray(input_data)

# reshape the numpy array
input_data_reshaped = input_data_as_numpy_array.reshape(1,-1)

# standardize the data
std_data = scaler.transform(input_data_reshaped)

prediction = model.predict(std_data)
print(prediction)


if (prediction[0] == 0).all():
  print("The Person does not have Parkinsons Disease")

else:

  print("The Person has Parkinsons")


## Using Convolutional Neural Networks

In [ ]:
new_data = np.array([199.22800,209.51200,192.09100,0.00241,0.00001,0.00134,0.00138,0.00402,0.01015,0.08900,0.00504,0.00641,0.00762,0.01513,0.00167,30.94000,0.432439,0.742055,-7.682587,0.173319,2.103106,0.068501
]).reshape(1, -1, 1)
new_data = scaler.transform(new_data.reshape(new_data.shape[0], new_data.shape[1]))
prediction = cnn_model.predict(new_data).round()
print(f'Prediction: {prediction[0][0]:.4f}')

if (prediction[0][0] == 0).all():
  print("The Person does not have Parkinsons Disease")

else:
  print("The Person has Parkinsons")


##Using Advanced Neural Networks

In [ ]:
new_data = np.array([199.22800,209.51200,192.09100,0.00241,0.00001,0.00134,0.00138,0.00402,0.01015,0.08900,0.00504,0.00641,0.00762,0.01513,0.00167,30.94000,0.432439,0.742055,-7.682587,0.173319,2.103106,0.068501
]).reshape(1, -1)
new_data = scaler.transform(new_data)
prediction = classifier.predict(new_data).round()
print(f'Prediction: {prediction[0][0]:.4f}')

if (prediction[0][0] == 0).all():
  print("The Person does not have Parkinsons Disease")

else:
  print("The Person has Parkinsons")
